# setup

In [ ]:
# coding: shift-jis

In [ ]:
import sys

print('defaultencoding:', sys.getdefaultencoding())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/10-23

In [ ]:
%pwd

In [ ]:
!apt install tesseract-ocr libtesseract-dev tesseract-ocr-jpn
!pip install pyocr

In [ ]:
!tesseract --list-langs

In [ ]:
from PIL import Image
import pyocr
import glob
import re
from tqdm import tqdm

import pandas as pd

# process

In [ ]:
tools = pyocr.get_available_tools()
tool = tools[0]
print("Will use tool '%s'" % (tool.get_name()))

In [ ]:
files = glob.glob("image/*")
print(files)

In [ ]:
df = pd.DataFrame(columns=[
                           'tel', 
                           'fax',
                           'mobile',
                           'organization',
                           'address',
                           'office',
                           'post',
                           'e-mail',
                           'homepage',
                           'file',
                           'other'
                           ])
# df = df.append({'tel': 0, 'fax': 1, 'mobile': 2, 'address': 3, 'e-mail': 5, 'other': 6}, ignore_index=True)
# df

In [ ]:
img = Image.open("image/スキャン 2021-10-23 12.48.jpg")
txt = tool.image_to_string(img, lang="jpn", builder=pyocr.builders.TextBuilder(tesseract_layout=3))
print(txt)

In [ ]:
img

In [ ]:
for file in tqdm(files):
    img = Image.open(file)
    txt = tool.image_to_string(img, lang="jpn", builder=pyocr.builders.TextBuilder(tesseract_layout=3))
    txt_list = txt.split('\n')
    raw = {'tel': '', 'fax': '', 'mobile': '', 'organization': '', 'address': '', 'e-mail': '', 'post': '','homepage': '', 'file': file, 'other': ''}
    for line in txt_list:
        choiced = False # いずれかのカラムにチョイスされたか
        # 前処理
        line = re.sub(r"[\u3000 \t]", "", line)
        line = line.strip()

        if ('連盟' or '会社' or '組' or '法人' or '有限' or '株式') in line:
            raw['organization'] += line
            choiced = True
        if ('TEL' or 'tel' or 'Tel' or '電話') in line:
            raw['tel'] += line
            choiced = True
        if ('FAX' or 'fax' or 'Fax') in line:
            raw['fax'] += line
            choiced = True
        if ('mob' or 'Mob' or 'MOB' or '携帯' or '電話') in line:
            raw['mobile'] += line
            choiced = True
        if ('mai' or 'Mai' or 'MAI' or '@' or 'メール') in line:
            raw['e-mail'] += line
            choiced = True
        if ('県' or '北海道' or '都' or '府' or '丁目' or '番地' or '市' or '自宅') in line:
            raw['address'] += line
            choiced = True
        if ('県' or '北海道' or '都' or '府' or '丁目' or '番地' or '市' or '事務' or '本社' or 'ビル') in line:
            raw['office'] += line
            choiced = True
        if ('〒' or 'post' or 'Post' or 'POST' or '郵便') in line:
            raw['post'] += line
            choiced = True
        if ('http' or 'home' or 'page' or 'ページ' or 'url' or 'URL' or 'Url' or 'link' or 'Link' or 'LINK') in line:
            raw['homepage'] += line
            choiced = True
        if choiced == False:
            raw['other'] += line

    df = df.append(raw, ignore_index=True)
    print(raw)
print(df)


In [ ]:
df

In [ ]:
df.to_csv('out/result1.csv', encoding='utf_8_sig')